In [1]:
import pandas as pd, numpy as np

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel\parentpoller.py:116: UserWarning: Parent poll failed.  If the frontend dies,
                the kernel may be left running.  Please let us know
                about your system (bitness, Python, etc.) at
                ipython-dev@scipy.org
  ipython-dev@scipy.org""")


In [2]:
dm=pd.read_excel('data/szekelyhon_medve.xlsx')
dc=pd.read_excel('data/szekelyhon_medve_curated.xlsx')

In [3]:
dc=dc[(dc['duplicate']!=1)&(dc['relevant']==1)]

In [4]:
dc["uid"] = dc["title"].map(str) + dc["content"]
dm["uid"] = dm["title"].map(str) + dm["content"]

In [5]:
dc=dc.set_index('uid')
dm=dm.set_index('uid')

In [6]:
dmi=dm.loc[dc.index.dropna()]

In [7]:
df=dmi.merge(dc).drop_duplicates()

Working df ready. Now infer location.

In [8]:
locs=df[['category','icategory','tags','title','content']]

In [9]:
locs

,category,icategory,tags,title,content
0,aktualis,udvarhely,"['Maros megye', 'medve']",Saját portájukon támadt rájuk a medve,Súlyos sérülésekkel került kórházba egy házasp...
1,aktualis,csik,"['Csíkszereda', 'medve']",Kétszer kellett közbelépniük a kórház mellett ...,A Csíkszeredai Megyei Sürgősségi Kórház külső ...
2,aktualis,udvarhely,"['Maros megye', 'medve']","Rátámadt a medve, amputálni kellett a lábát",Amputálni kellett a bal lábát annak a 28 éves ...
3,aktualis,udvarhely,"['Maros megye', 'medve']",Nem sietik el a kilövéseket,Tizenöt medve kiemelését hagyták jóvá idén Mar...
4,aktualis,csik,"['Csíkszentgyörgy', 'medve']",Éjszakai harc a medvével Csíkszentgyörgyön,Hiába próbálta elkergetni a medvét vasvillával...
5,vilag,udvarhely,"['Belföld', 'medve']",Újabb medvét gázoltak el az autópályán,Újabb medvét gázoltak el szerdán az A1-es autó...
6,aktualis,csik,"['Csíkdánfalva', 'medve']","Hatástalannak bizonyult a villanypásztor, több...",Három borjút és több juhot pusztított el egy m...
7,aktualis,udvarhely,"['medve', 'Udvarhelyszék']",A székelyderzsi erődtemplomban kérték számon R...,A elszaporodott medvepopuláció miatti károk és...
8,vilag,csik,"['Belföld', 'medve']","Fejszével vágta fejbe a medvét, hogy feleségét...",Állataikat akarta védeni a medvétől az az assz...
9,aktualis,csik,['Csíkszentmihály'],Három medvét gázolt el a vonat Lóvészen,Három medvét gázolt el szombat reggel a vonat ...


In [10]:
import json, requests

In [11]:
r=requests.get('http://szekelydata.csaladen.es/lakossag/dict.json')

In [12]:
lak=json.loads(r.content)

In [13]:
lak={lak[i][3]:lak[i][2] for i in lak}

In [14]:
#osszes erdelyi magyar telepulesnev
laks=lak.keys()

In [16]:
def levenshteinDistance(s1, s2):
    if len(s1) > len(s2):
        s1, s2 = s2, s1

    distances = range(len(s1) + 1)
    for i2, c2 in enumerate(s2):
        distances_ = [i2+1]
        for i1, c1 in enumerate(s1):
            if c1 == c2:
                distances_.append(distances[i1])
            else:
                distances_.append(1 + min((distances[i1], distances[i1 + 1], distances_[-1])))
        distances = distances_
    return distances[-1]

In [17]:
#look for perfect matches
decoded_locs={}
for i in locs.index:
    if '[' in locs.loc[i]['tags']:
        k=locs.loc[i]['tags'].replace("'",'"')
        for j in json.loads(k):
            if j in laks:
                if i not in decoded_locs:decoded_locs[i]=[]
                decoded_locs[i].append(j)
                print('1',k,j)
    for j in locs.loc[i]['title'].split():
        if j in laks:
            if i not in decoded_locs:decoded_locs[i]=[]
            decoded_locs[i].append(j)
            print('2',j)
    for j in locs.loc[i]['content'].split():
        if j in laks:
            if i not in decoded_locs:decoded_locs[i]=[]
            decoded_locs[i].append(j)
            print('3',j)
    for j in locs.loc[i]['title'].split():
        for k in laks:
            if levenshteinDistance(j,k)<3:
                if i not in decoded_locs:decoded_locs[i]=[]
                decoded_locs[i].append(k)
                print('4',j,k)
    for j in locs.loc[i]['content'].split():
        for k in laks:
            if levenshteinDistance(j,k)<3:
                if i not in decoded_locs:decoded_locs[i]=[]
                decoded_locs[i].append(k)
                print('5',j,k)

3 Görgényhodák
5 Görgényhodák Görgényhodák
1 ["Csíkszereda", "medve"] Csíkszereda
4 csíkszeredai Csíkszereda
5 Csíkszeredai Csíkszereda
5 két Réty
3 Bala
5 bal Bala
5 Bala Bala
5 hét Réty
5 négy Réty
1 ["Csíkszentgyörgy", "medve"] Csíkszentgyörgy
4 Csíkszentgyörgyön Csíkszentgyörgy
5 csíkszentgyörgyi Csíkszentgyörgy
5 sávon Zágon
1 ["Csíkdánfalva", "medve"] Csíkdánfalva
5 csíkdánfalvi Csíkdánfalva
4 székelyderzsi Székelyderzs
5 Székelyderzsbe Székelyderzs
1 ["Csíkszentmihály"] Csíkszentmihály
3 Csíkszentmihály
3 Csíkszentmihály
5 Csíkszentmihály Csíkszentmihály
5 Csíkszentmihály Csíkszentkirály
5 Csíkszentmihály Csíkszentmihály
5 Csíkszentmihály Csíkszentkirály
3 Marosvásárhely
3 Gyergyószentmiklós
5 Székelyudvarhelyen Székelyudvarhely
5 Marosvásárhely Marosvásárhely
5 Gyergyószentmiklós Gyergyószentmiklós
1 ["Csíkszereda", "medve"] Csíkszereda
1 ["medve", "Segesvár"] Segesvár
4 segesvári Segesvár
5 két Réty
5 Maria Maksa
2 Székelykeresztúr
4 Székelykeresztúr Székelykeresztúr
1 ["medve

In [18]:
predicted_locs=[]
for i in range(len(locs.index)):
    if locs.index[i] in decoded_locs: 
        dummy=list(set(decoded_locs[locs.index[i]]))
        if len(dummy)==1:
            dummy=dummy[0]
        else:
            dummy=repr(decoded_locs[locs.index[i]])
        predicted_locs.append(dummy)
    else: predicted_locs.append('')

In [19]:
locs['predicted_locs']=predicted_locs
locs['actual_locs']=predicted_locs

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [20]:
lm=locs[['title','content','predicted_locs','actual_locs']]

In [21]:
lm.to_excel('data/szekelyhon_locs.xlsx')
lm.to_excel('data/szekelyhon_locs_curated.xlsx')

Open `szekelyhon_locs_curated` and manually verify predicted locations. Fill where necessary.